In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

In [58]:
!conda install lxml

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h7b6447c_0         3.8 MB
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB
    certifi-2020.4.5.1         |           py36_0         159 KB
    ------------------------------------------------------------
                                           Total:         5.5 MB

The following packages will be UPDATED:

    certifi:         2020.4.5.1-py36h9f0ad1d_0 conda-forge --> 2020.4.5.1-py36_0   
    lxml:            4.3.1-py36hefd8a0e_0                  --> 4.5.0-py36hefd8a0e_0
    openssl:         1.1.1f-h516909a_0         conda-forge --> 1.1.1f-h7b6447c_0   

The following packages will be DOWNGRADED:

    ca-certificates: 2020.4.5.1-hecc5488_0     conda-for

In [4]:
##SCRAPPING DISTRICS AND NEIGHBORHOODS OF MADRID

#define headers for the request
headers = requests.utils.default_headers()
headers.update({"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"})

#Wikipedia link for the information to scrap
url = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [148]:
#Finding columns name html style
columns_name = str(soup.center.table.tbody.tr).replace('\n','')

#cleaning tags
columns_string=re.findall('<th>(.+?)<',columns_name)

columns = [list(columns_string)[1],list(columns_string)[3]]
#Transform in a Series to create data frame
columns_name = pd.Series(list(columns))

In [44]:
#finding dataframe values
values = str(soup.center.find_all("td")).replace('\n','')
values = re.findall('>(.+?)</td>',values)

In [45]:
#finding Neighbourhood clean name
neigh = []
for value in values:
    if len(re.findall('>(.+?)</a>',str(value)))<1:
        neigh.append(value)
    else:
        a = re.findall('>(.+?)<',str(value)) #result type is a list
        neigh.append(a[0])


In [145]:
#Define two empty list to get District Name and population
d=[]
h1=[]
h2=[] #h1 will return a nest list, to extract the population in just one list
i=0

for i in range(int(len(neigh)/7)):
    if i==0:
        d.append(neigh[i+1])
        h1.append(re.findall('>(.+?)</span>',str(neigh[i+3])))
        h2.append(h1[i][i])
    else:
        d.append(neigh[i*7+1])
        h1.append(re.findall('>(.+?)</span>',neigh[i*7+3]))
        h2.append(h1[i][0])


In [154]:
#Creating the df wich contains info about Madrid districts

dicti = {columns_name[0]:d,
         columns_name[1]:h2    
}

madrid = pd.DataFrame(dicti)
madrid

,Name,Population
0,Centro,"131,928"
1,Arganzuela,"151,965"
2,Retiro,"118,516"
3,Salamanca,"143,800"
4,Chamartín,"143,424"
5,Tetuán,"153,789"
6,Chamberí,"137,401"
7,Fuencarral-El Pardo,"238,756"
8,Moncloa-Aravaca,"116,903"
9,Latina,"233,808"


In [106]:
147/7

21.0

None


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Districts of Madrid - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"Xo5MAwpAML8AAajyTRAAAAAS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Districts_of_Madrid","wgTitle":"Districts of Madrid","wgCurRevisionId":932631329,"wgRevisionId":932631329,"wgArticleId":10147562,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 Spanish-language sources (es)","Articles with short description","Articles that may contain original research from March 2014","All articles that may contain 